# Naive Bayes Classifier in Numpy - Lab

## Introduction
Naive Bayes classifiers are a **family of probabilistic classifiers** based on Bayes' theorem, with an assumption that each feature is independent from one another. Previously we looked at the learning mechanism of this classifier in terms of maximizing the posterior probability. In this lab we shall learn to code a **Gaussian Naive Bayes Classifier** from scratch, and also learn to use scikitlearn library for this task.   

## Objectives

You will be able to:

- Build a Naive Bayes Classifier in Python and Numpy to make predictions on unseen data 

## Building a Gaussian NB Classifier

Below we shall attempt to build a naive Bayes classifier in using numpy calculations only. Python offers sophisticated implementations of this algorithm in SciKitLearn which we shall look at in the following lesson. Here we will use the equations we have learned so far, and put them into action for a very simple example. 

## Problem:  Gender Classification 

Let's work with a small toy-data with continuous features (height, weight, foot size) and a target variable (Person: male or a female). We would work on building a classifier that can learn the joint probability of data and the target variables and classify a new example as a male of female. 

Note: You may also use a multinomial distribution for footsize (categrocial). Let's just assume they are all continuous for now.

### Read the data `gender.csv` into Pandas data frame and inspect its content. 

In [8]:
import numpy as np
import pandas as pd
data = pd.read_csv('gender.csv')
data
# 	Person	height	weight	foot size
# 0	male	6.00	180	12
# 1	male	5.92	190	11
# 2	male	5.58	170	12
# 3	male	5.92	165	10
# 4	female	5.00	100	6
# 5	female	5.50	150	8
# 6	female	5.42	130	7
# 7	female	5.75	150	9

,Person,height,weight,foot size
0,male,6.00,180,12
1,male,5.92,190,11
2,male,5.58,170,12
3,male,5.92,165,10
4,female,5.00,100,6
5,female,5.50,150,8
6,female,5.42,130,7
7,female,5.75,150,9


So a very small dataset, but this will help us better understand how the classifier works in more detail. The results surely wont be groundbreaking. We can see that gender is shown as strings male/female . Let's change "male" to 0 and "female" to 1 and make a binary categorical variable. 

### Index the labels $[male, female] \rightarrow [0, 1] $.

In [9]:
# Subset data and assign 0 and 1 
data['Person'] = data['Person'].replace({'male': 0, 'female': 1})
data
# Person	height	weight	foot size
# 0	0	6.00	180	12
# 1	0	5.92	190	11
# 2	0	5.58	170	12
# 3	0	5.92	165	10
# 4	1	5.00	100	6
# 5	1	5.50	150	8
# 6	1	5.42	130	7
# 7	1	5.75	150	9

,Person,height,weight,foot size
0,0,6.00,180,12
1,0,5.92,190,11
2,0,5.58,170,12
3,0,5.92,165,10
4,1,5.00,100,6
5,1,5.50,150,8
6,1,5.42,130,7
7,1,5.75,150,9


This is great. Now that we have our data in the format that we need, we can start focusing on the Naive Bayes equation and take our experiment forward. 

We have our data data $x = (x_{1}, \ldots, x_{n})$ (height, weight, footsize) to be classified, Using joint probabilities between data and target class (Person), NB classifier assigns a discrete probability to all examples of feature data as shown below:

$$ p(C_{k}|x_{1}, \ldots, x_{n} )$$

Here, $K$ represents the numbers on classes in the target variable. 

### Gaussian PDF
With continuous data, we assume that the features for each class are distributed according to a Normal/Gaussian distribution and class probability can be calculated using Gaussian PDF function below

$$ f(x) = \dfrac{1}{\sqrt{2 \pi \sigma^2}} \cdot exp\bigg(\dfrac{-(x - \mu)^2}{2 \sigma^2}\bigg) $$

Where $\sigma^2$ represents the variance of the values in $x$, while $\mu$ represents the mean of the values in $x$. So to calculate the distribution function, we first need to calculate the mean and variance of each variable with respect to the target classes (male or female). 

### Segment (subset) the dataset by class (male/female) and calculate the mean and var of features for male and female classes. 

Before we can compute for the probability distribution for features $x$, we must first compute for the mean $\mu$ and variance $\sigma^{2}$ values of $x_{i}$ for each $k$ class.

In [24]:
# Your code here 
males = data.loc[data['Person'] == 0]
females = data.loc[data['Person'] == 1]
x_cols = ['height', 'weight', 'foot size']

male_means = males[x_cols].mean()
male_vars = males[x_cols].var()
female_means = females[x_cols].mean()
female_vars = females[x_cols].var()

print(f'Mean Values for Male Features\n{male_means}\n')
print(f'Variance Values for Male Features\n{male_vars}\n')
print(f'Mean Values for Female Features\n{female_means}\n')
print(f'Variance Values for Female Features\n{female_vars}\n')

# Example output

# Mean values for male features
# height         5.855
# weight       176.250
# foot size     11.250
# dtype: float64

# Variance values for male features
# height         0.035033
# weight       122.916667
# foot size      0.916667
# dtype: float64

# Mean values for female features
# height         5.4175
# weight       132.5000
# foot size      7.5000
# dtype: float64

# Variance values for female features
# height         0.097225
# weight       558.333333
# foot size      1.666667
# dtype: float64

Mean Values for Male Features
height         5.855
weight       176.250
foot size     11.250
dtype: float64

Variance Values for Male Features
height         0.035033
weight       122.916667
foot size      0.916667
dtype: float64

Mean Values for Female Features
height         5.4175
weight       132.5000
foot size      7.5000
dtype: float64

Variance Values for Female Features
height         0.097225
weight       558.333333
foot size      1.666667
dtype: float64



### Decision Rule

In order to build a functional classifier from the model above, we need some kind of a decision rule, (this applies to all classifiers). For our NB classifier, we use the $ARGMAX$ function for identifying a unknown target class for a new example.

$$ \hat{y} = argmax_{k \in \{1,\ldots,K\}} \bigg(p(C_{k}) \prod_{i=1}^{n} p(x_{i}|C_{k})\bigg) $$

Now that we have the $\mu$ and $\sigma^{2}$ values for each features $x_{i}$ per $k$-class, let us now write a function for our $likelihood$ computation, i.e. 

$$p(x_{i}|C_{k})$$. 

Recall that we are going to plugin the likelihood computation into the Gaussian probability density function,

$$ p(x = x_{i} | C_{k}) = \dfrac{1}{\sqrt{2 \pi \sigma_{k}^{2}}} \cdot exp\bigg(\dfrac{-(x_{i} - \mu_{k})^2}{2 \sigma_{k}^{2}}\bigg) $$

### Implement the `likelihood(xi, mu, var)` function from above equation

In [28]:
def likelihood(xi, mu, var):
    den1 = np.sqrt(2 * np.pi * var)
    den2 = 2 * var
    num1 = 1
    num2 = -(xi - mu) ** 2
    return (num1 / den1) * np.exp(num2 / den2)

So this is our function for computing likelihood. We shall now compute the $prior$ probability for $k$-classes. 

There are two ways to do this: 
- Give an equal probability for each $k$-classes i.e. a uniform prior
- (number of class samples) / (total number of samples). 

For this small dataset, we have equal number of classes , and both approaches will lead us to have . uniform prior anyway. we shall get a prior probabiity of 0.5 since there are exactly 4 samples for each class.

### Declare prior probability of both target classes

In [31]:
class_priors = np.array([len(males) / len(data), len(females) / len(data)])
class_priors


# array([0.5, 0.5])

array([0.5, 0.5])

Great, with our `class_priors` array $p(C_{k})$, we have everything we need to solve the equation.

### Classify a new example

We can now classify an un-labeled data. 

### Add a new example with features (but no target) to our dataset. 

In [56]:
# Add a new example with features of your choice (keep them reasonable)

data.loc[8] = [-99, 6, 130, 8]
data

# Person	height	weight	foot size
# 0	0	6.00	180	12
# 1	0	5.92	190	11
# 2	0	5.58	170	12
# 3	0	5.92	165	10
# 4	1	5.00	100	6
# 5	1	5.50	150	8
# 6	1	5.42	130	7
# 7	1	5.75	150	9
# 8	-99	6.00	130	8 <-- new example

,Person,height,weight,foot size
0,0.0,6.00,180.0,12.0
1,0.0,5.92,190.0,11.0
2,0.0,5.58,170.0,12.0
3,0.0,5.92,165.0,10.0
4,1.0,5.00,100.0,6.0
5,1.0,5.50,150.0,8.0
6,1.0,5.42,130.0,7.0
7,1.0,5.75,150.0,9.0
8,-99.0,6.00,130.0,8.0


> Using value -99 or similar values for unknowns in a dataset is a common way of processing data without generating Nan errors , yet keep the unknown identifiable by putting in some unlikely value i.e. -99 or -1 , which can later be searched for.

Let's calculate likelihood (probability of unknown class given new data) of each xi for the new example. 

### Using` likelihood()`, get likelihood values  `height`, `weight` and `footsize`  for new example. 

In [59]:
# height feature
x_1 = (likelihood(data.loc[8]['height'], male_means[0], male_vars[0]), 
       likelihood(data.loc[8]['height'], female_means[0], female_vars[0]))
# weight feature
x_2 = (likelihood(data.loc[8]['weight'], male_means[1], male_vars[1]), 
       likelihood(data.loc[8]['weight'], female_means[1], female_vars[1]))
# foot size feature
x_3 = (likelihood(data.loc[8]['foot size'], male_means[2], male_vars[2]), 
       likelihood(data.loc[8]['foot size'], female_means[2], female_vars[2]))

x_1, x_2, x_3

# (array([1.57888318, 0.22345873]),
#  array([5.98674302e-06, 1.67892979e-02]),
#  array([0.00131122, 0.2866907 ]))

((1.5788831832641037, 0.223458726844816),
 (5.986743024812142e-06, 0.01678929788990836),
 (0.0013112210442223463, 0.286690699891019))

This completes our Gaussian likelihood $p(x=x_{1} | C_{k})$. 

Now that we have all the likelihood values and our prior probabilities, the variables in our equation are now complete. Now we need to calculate the formula 
$$p(C_{k}) \prod_{i=0}^{n} p(x_{i}|C_{k})$$

In this particular example , it can be written simply as:

$$ prediction = prior \times x_{1} \times x_{2} \times x_{3} $$ 


### Calculate the class prediction for new example using above formula

In [64]:
prediction = class_priors * x_1 * x_2 * x_3
prediction

array([6.19707184e-09, 5.37790918e-04])

### Normalizing with "Evidence" P(X)

Recall that Gaussian $p(x_{i} | C_{k})$ equation gives a probability density, not a probability distribution. To get the equivalent probability distribution, we  need to **normalize** the probability density. Recall the denominator in Bayes laws , also known as "Probability of Data" or "Evidence" is calculated as below:


$$ \sum_{i = 0}^{k - 1} \Bigg( p(c_{i}) \prod_{j = 0}^{n - 1} p(x_{j} | c_{i}) \Bigg) $$

Concretely, the `evidence` may be computed as follows in this case,

$$ evidence = {p(male)\ p(height|male)\ p(weight|male)\ p(foot\ size|male)\\ +\ p(female)\ p(height|female)\ p(weight|female)\ p(foot\ size|female)} $$

> The `evidence` is the sum of all joint probability $p(C_{k}, x)$.



### Normalize the predictions using the equation for `evidence` we have defined above

In [65]:
evidence = sum(prediction)
posterior = prediction/evidence

In [66]:
posterior
# array([1.15230663e-05, 9.99988477e-01])

array([1.15230663e-05, 9.99988477e-01])

#### Sanity check

The `posterior` probability values SHOULD now sum up to 1, i.e. a probability distribution. Let's check for it. 

In [67]:
# Uncomment to check 
np.sum(posterior)

# 1.0

1.0

### Bring in `ARGMAX` , the decision rule

So now we have posterior class probabilities for each class, that sum up to 1. So naturally, which ever class shows a higher probability, will be chosen as the prediction. 

$$ \hat{y} = argmax_{k \in \{1,\ldots,K\}} \bigg(p(C_{k}) \prod_{i=1}^{n} p(x_{i}|C_{k})\bigg) $$

### To get our predicted class, use the $np.argmax()$ function with `posterior`


In [68]:
# Predict the class using argmax
np.argmax(posterior)

# The Naive Bayes predicts Class: 1

1

Recall that the index `1` refers to `female`, hence our classifer predicts that a probable class for new example is `female`. Neat isn't it. Pretty naive , yet highly effective.

## Level Up - Optional 

- Read the dataset `diabetes.csv` into your code and modify the code it to perform predictions on presence or absence of diabetes using a number of available features and a target variable. 

You may need to covert some of the code into functions to help you process your data faster as this now has 8 features. 

In [70]:
diab = pd.read_csv('diabetes.csv')
diab.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [79]:
neg = diab.loc[diab['Outcome'] == 0]
pos = diab.loc[diab['Outcome'] == 1]
cols = list(diab.columns[:-1])

pos_means = pos[cols].mean()
pos_vars = pos[cols].var()
neg_means = neg[cols].mean()
neg_vars = neg[cols].var()

print(f'Mean Values for Positive Outcome\n{pos_means}\n')
print(f'Variance Values for Positive Outcome\n{pos_vars}\n')
print(f'Mean Values for Negative Outcome\n{neg_means}\n')
print(f'Variance Values for Negative Outcome\n{neg_vars}\n')

Mean Values for Positive Outcome
Pregnancies                   4.865672
Glucose                     141.257463
BloodPressure                70.824627
SkinThickness                22.164179
Insulin                     100.335821
BMI                          35.142537
DiabetesPedigreeFunction      0.550500
Age                          37.067164
dtype: float64

Variance Values for Positive Outcome
Pregnancies                    13.996870
Glucose                      1020.139457
BloodPressure                 461.897968
SkinThickness                 312.572195
Insulin                     19234.673319
BMI                            52.750693
DiabetesPedigreeFunction        0.138648
Age                           120.302588
dtype: float64

Mean Values for Negative Outcome
Pregnancies                   3.298000
Glucose                     109.980000
BloodPressure                68.184000
SkinThickness                19.664000
Insulin                      68.792000
BMI                          3

In [80]:
class_priors = np.array([len(pos) / len(diab), len(neg) / len(diab)])
class_priors

array([0.34895833, 0.65104167])

In [86]:
diab.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [90]:
# diab.drop(diab.tail(1).index,inplace=True)
diab.loc[diab.shape[0]] = [0, 126, 92, 17, 84, 28.7, .45, 34, -99]
diab.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,1.0
767,1.0,93.0,70.0,31.0,0.0,30.4,0.315,23.0,0.0
768,0.0,126.0,92.0,17.0,84.0,28.7,0.450,34.0,-99.0


In [92]:
# Pregnancies feature
preg = (likelihood(diab.loc[768]['Pregnancies'], pos_means[0], pos_vars[0]), 
       likelihood(diab.loc[768]['Pregnancies'], neg_means[0], neg_vars[0]))
# Glucose feature
gluc = (likelihood(diab.loc[768]['Glucose'], pos_means[1], pos_vars[1]), 
       likelihood(diab.loc[768]['Glucose'], neg_means[1], neg_vars[1]))
# BloodPressure feature
x_3 = (likelihood(data.loc[8]['foot size'], male_means[2], male_vars[2]), 
       likelihood(data.loc[8]['foot size'], female_means[2], female_vars[2]))

preg

(0.04577250710378575, 0.07275440128565916)

In [96]:
i=0
for col in cols:
    exec(col + " = (likelihood(diab.loc[768]['Pregnancies'], pos_means[i], pos_vars[i]), likelihood(diab.loc[768]['Pregnancies'], neg_means[i], neg_vars[i]))")
    i += 1

Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age

((0.04577250710378575, 0.07275440128565916),
 (7.06699800968648e-07, 2.188016835175787e-06),
 (8.136873597198561e-05, 1.778309984453126e-05),
 (0.010283896440068021, 0.011202180922871823),
 (0.0022141880265826646, 0.0031676149924054525),
 (4.5284557116522416e-07, 2.201464357588305e-05),
 (0.3591888282967095, 0.4751385822032259),
 (0.00012043039376446595, 0.0009598208600658433))

In [98]:
prediction = class_priors * Pregnancies * Glucose * BloodPressure * SkinThickness * Insulin * BMI * DiabetesPedigreeFunction * Age
prediction

array([4.09686881e-28, 6.56573415e-25])

In [99]:
evidence = sum(prediction)
posterior = prediction/evidence
posterior

array([6.23588156e-04, 9.99376412e-01])

In [102]:
prediction[0] / evidence, prediction[1] / evidence

(0.0006235881557856036, 0.9993764118442144)

In [101]:
np.argmax(posterior)

1

## Summary 

In this lab, we looked at building a Naive Bayes classifier from scratch. This was not a complete machine learning experiment as we rather focused on the seeing how the algorithm performs in relation to underlying mathematics. NExt we shall look at how to achieve this functionality in SciKitLearn. 